In [1]:
import pandas as pd

In [2]:
import os

In [27]:
os.getcwd()

'C:\\Users\\chris\\data_science_projects\\data-question-4-affordable-housing-green-dogs'

In [29]:
bp = pd.read_csv('Building_Permits_Issued.csv')

C:\Users\chris\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
bp

,Permit #,Permit Type Description,Permit Subtype Description,Parcel,Date Entered,Date Issued,Const. Cost,Address,City,State,Zip,Subdivision/Lot,Contact,Permit Type,Permit Subtype,IVR Trk#,Purpose,Council Dist,Census Tract,Mapped Location
0,2018002927,Building Residential - New,Single Family Residence,081160R00100CO,01/18/2018,02/12/2018,206150.0,1017A MONROE ST,NASHVILLE,TN,37208,UNIT A 1017 MONROE,"TRISTAR CONSTRUCTION, INC",CARN,CAA01R301,3439281,to construct 1830SF single family residence wi...,19.0,37019400.0,"1017A MONROE ST\nNASHVILLE, TN 37208\n(36.1740..."
1,201617929,Building Use & Occupancy,"Tents, Stages",09215014800,04/19/2016,04/21/2016,2000.0,2300 WEST END AVE,NASHVILLE,TN,37203,N W CORNER WEST END AVE & 23RD AVE N,Music City Tents & Events Llc,CAUO,CAZ06A001,2116751,PERMIT TO ERECT 1:40X85 TENT. THIS TENT IS BEI...,21.0,37016500.0,"2300 WEST END AVE\nNASHVILLE, TN 37203\n(36.14..."
2,2016049270,Building Sign Permit,Sign - Ground / Wall Signs,11909014000,09/14/2016,09/14/2016,15000.0,2802 NOLENSVILLE PIKE,NASHVILLE,TN,37211,PT LOT 2 PATTERSON SUB,BOZMAN SIGN CO INC,CASN,CAZ02A001,3277064,to install replacement signage for existing fr...,16.0,37017300.0,"2802 NOLENSVILLE PIKE\nNASHVILLE, TN 37211\n(3..."
3,2016029907,Building Residential - Roofing / Siding,Duplex,075150A13900CO,06/20/2016,06/20/2016,17648.0,139 STONERS GLEN DR,HERMITAGE,TN,37076,UNIT 139 STONERS GLEN PHASE 1,"MR ROOF NASHVILLE, LLC",CARK,CAA02R302,3251947,Remove existing shingles and replace with new ...,12.0,37015623.0,"139 STONERS GLEN DR\nHERMITAGE, TN 37076\n(36...."
4,201533533,Building Residential - Tenant Finish Out,"Multifamily, Tri-Plex, Quad, Apartments",10404024100,08/07/2015,11/17/2015,542696.0,70 MUSIC SQ W,NASHVILLE,TN,37203,LOT 1 O.B. HAYES FIRST PLAN CONSOLIDATION OF L...,Construction Enterprises Inc,CART,CAA03R398,2061885,SEE MASTER PERMIT T2015-26520...T.....GRAND AV...,19.0,37016400.0,"70 MUSIC SQ W\nNASHVILLE, TN 37203\n(36.147074..."
5,201605753,Building Residential - Rehab,Single Family Residence,131060E32200CO,02/10/2016,02/10/2016,50000.0,322 BOXMERE PL,NASHVILLE,TN,37215,UNIT 322 BURTON HILLS VILLAGE OF BOXMERE-PHASE 1,Sharp & Robbins Construction Llc,CARR,CAA01R301,2100747,to conduct interior renovation to existing uni...,25.0,37017902.0,"322 BOXMERE PL\nNASHVILLE, TN 37215\n(36.09425..."
6,2017032777,Building Residential - New,Single Family Residence,072103B00300CO,05/17/2017,05/24/2017,226201.0,1037 FAIRWIN AVE,NASHVILLE,TN,37216,UNIT 3 2619 GALLATIN PIKE TOWNHOMES,PARAGON GROUP LLC,CARN,CAA01R301,3367050,to construct 2008Sf single family residence wi...,5.0,37011400.0,"1037 FAIRWIN AVE\nNASHVILLE, TN 37216\n(36.198..."
7,201546095,Building Residential - Rehab,Single Family Residence,12809002600,10/22/2015,10/22/2015,30000.0,305 HARPETH VALLEY RD,NASHVILLE,TN,37221,LOT 35 B. V. 1776 ESTATES,"Odyssey Construction, Llc",CARR,CAA01R301,2078541,to make general repairs to existing residence....,22.0,37018301.0,"305 HARPETH VALLEY RD\nNASHVILLE, TN 37221\n(3..."
8,201619676,Building Commercial - Roofing / Siding,"Manufacturing, Light Hazardous (H1-H5)",09314033300,04/28/2016,04/28/2016,66961.0,625 FOGG ST,NASHVILLE,TN,37203,LOTS 30 TH 34 EWING & WETMORE,Ragan Construction Co Inc (Formerly Southern R...,CACK,CAH06H000,2119017,to reroof existing cromatics building.,19.0,37019500.0,"625 FOGG ST\nNASHVILLE, TN 37203\n(36.150909, ..."
9,201550728,Building Commercial - Foundation,"Multifamily, Apt / Twnhome > 5 Unit Bldg",08305009600,11/18/2015,02/01/2016,219566.0,1035 W EASTLAND AVE,NASHVILLE,TN,37206,"LOT 4 EDGEFIELD HOSPITAL, INC. RESUB",Southland Constructors Inc.,CACF,CAA03R399,2084584,"per shell permit 201550039, to conduct foundat...",5.0,37011700.0,"1035 W EASTLAND AVE\nNASHVILLE, TN 37206\n(36...."


In [ ]:
sf2009=
sf2013=
sf2017=